In [1]:
import pandas as pd
import numpy as np

In [2]:
tracking_test = pd.read_json("Player_Tracking/CLEBOS/2017051902_nba-bos_TRACKING.jsonl.gz", lines=True,
                             compression='gzip', encoding='latin1')

In [3]:
events_test = pd.read_json("Player_Tracking/CLEBOS/2017051902_nba-bos_EVENTS.jsonl", lines=True)

In [4]:
events_test.eventType.unique()

array(['TOUCH', 'DRIBBLE', 'PASS', 'SHOT', 'REB', 'FOUL', 'TO', 'FT',
       'VIO', 'TMO'], dtype=object)

In [5]:
temp = tracking_test.merge(events_test[['eventType','playerId','wallClock','gameClock']], how='left', on=['wallClock','gameClock'])

In [6]:
temp.dropna(subset=['eventType','playerId'], how='all', inplace=True)

In [7]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3603 entries, 147 to 84892
Data columns (total 11 columns):
awayPlayers         3603 non-null object
ball                3603 non-null object
frameIdx            3603 non-null int64
gameClock           3603 non-null float64
gameClockStopped    3603 non-null bool
homePlayers         3603 non-null object
period              3603 non-null int64
shotClock           3549 non-null float64
wallClock           3603 non-null int64
eventType           3603 non-null object
playerId            3574 non-null object
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 313.2+ KB


In [8]:
from ss_preprocess import *

In [9]:
change_coord_format(temp)
label_possession(temp)
label_possession_idx(temp)
label_half_court(temp)
convert_coordinates(temp)

In [10]:
temp.shape

(1847, 15)

In [11]:
temp.head()

,awayPlayers,ball,frameIdx,gameClock,gameClockStopped,homePlayers,period,shotClock,wallClock,eventType,playerId,poss_player,poss_team,poss_idx,half_court_idx
289,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [[15....,"[[0.8999999999999999, 468.0]]",289,713.42,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [[93....,1,18.65,1495240353096,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
315,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [[105...,"[[74.6, 514.3]]",315,712.38,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [[97....,1,17.62,1495240354136,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
330,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [[129...,"[[159.8, 551.1]]",330,711.77,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [[84....,1,17.03,1495240354736,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
344,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [[141...,"[[192.8, 617.3]]",344,711.21,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [[52....,1,16.47,1495240355296,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
371,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [[151...,"[[154.70000000000002, 687.8]]",371,710.13,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [[-10...,1,15.41,1495240356376,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0


In [12]:
temp.loc[289,'awayPlayers']

{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [array([ 15.4, 870. ])],
 'ff403f7d-89ef-11e6-b44d-a45e60e298d3': [array([ 50.4, 753.2])],
 'ff42aecf-89ef-11e6-8cb8-a45e60e298d3': [array([ 62.4, 656.9])],
 'ff4221b5-89ef-11e6-b799-a45e60e298d3': [array([ -6.9, 557. ])],
 'ff41f1fa-89ef-11e6-a048-a45e60e298d3': [array([ 91.5, 699.2])]}